In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

2024-07-07 15:19:22.401478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load datasets
alphabets_df = pd.read_csv('alphabets_28x28.csv')
sentiment_df = pd.read_csv('sentiment_analysis_dataset.csv')
target_labels = pd.read_csv('target_labels.csv')

/var/folders/pn/ckbj4p2n0jsdrxj9rs94ljf00000gn/T/ipykernel_1934/3333406404.py:2: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  alphabets_df = pd.read_csv('alphabets_28x28.csv')


In [3]:
# OCR Model Training
def train_ocr_model():

    # Separate label column and pixel columns
    labels = alphabets_df['label']
    pixel_columns = alphabets_df.drop(columns=['label'])

    # Convert pixel columns to numeric
    pixel_columns = pixel_columns.apply(pd.to_numeric, errors='coerce')

    # Combine labels and pixel columns back
    alphabets_df_processed = pd.concat([labels, pixel_columns], axis=1)

    # Drop rows with any NaNs
    alphabets_df_processed = alphabets_df_processed.dropna()

    # Drop duplicates
    alphabets_df_processed = alphabets_df_processed.drop_duplicates()

    # Only keep rows with valid labels (A-Z)
    valid_labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    alphabets_df_processed = alphabets_df_processed[alphabets_df_processed['label'].isin(valid_labels)]

    if not alphabets_df_processed.empty:
        # Prepare data for OCR model
        X = alphabets_df_processed.drop(columns=['label']).values
        y = pd.get_dummies(alphabets_df_processed['label']).values

        # Split into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Reshape for CNN
        X_train = X_train.reshape(-1, 28, 28, 1)
        X_test = X_test.reshape(-1, 28, 28, 1)

        # Build OCR model
        model = Sequential([
            Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(len(valid_labels), activation='softmax')
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

        # Evaluate model
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"OCR Model Accuracy: {accuracy}")

        return model
    else:
        print("No data available for training the OCR model.")
        return None

In [4]:
# Load and preprocess target images for OCR model
def preprocess_target_images(target_images_dir):
    target_images_files = [os.path.join(target_images_dir, file) for file in os.listdir(target_images_dir) if file.endswith('.png')]
    target_images = []

    for file in target_images_files:
        image = load_img(file, color_mode='grayscale', target_size=(28, 28))
        image = img_to_array(image)
        target_images.append(image)

    target_images = np.array(target_images)
    return target_images, target_images_files

In [5]:
# Train OCR model
ocr_model = train_ocr_model()

/Users/manasvigampa/Documents/Vscode/myenv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
5026/5026 ━━━━━━━━━━━━━━━━━━━━ 162s 32ms/step - accuracy: 0.7916 - loss: 1.3847 - val_accuracy: 0.9564 - val_loss: 0.1589
Epoch 2/5
5026/5026 ━━━━━━━━━━━━━━━━━━━━ 257s 43ms/step - accuracy: 0.9615 - loss: 0.1349 - val_accuracy: 0.9579 - val_loss: 0.1518
Epoch 3/5
5026/5026 ━━━━━━━━━━━━━━━━━━━━ 141s 28ms/step - accuracy: 0.9725 - loss: 0.0939 - val_accuracy: 0.9610 - val_loss: 0.1534
Epoch 4/5
5026/5026 ━━━━━━━━━━━━━━━━━━━━ 152s 30ms/step - accuracy: 0.9793 - loss: 0.0699 - val_accuracy: 0.9606 - val_loss: 0.1622
Epoch 5/5
5026/5026 ━━━━━━━━━━━━━━━━━━━━ 155s 31ms/step - accuracy: 0.9830 - loss: 0.0583 - val_accuracy: 0.9650 - val_loss: 0.1714
1257/1257 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9659 - loss: 0.1675
OCR Model Accuracy: 0.9649786353111267


In [6]:
if ocr_model:
    target_images_dir = 'target_images'
    target_images, target_images_files = preprocess_target_images(target_images_dir)

    # Predict on target images
    target_predictions = ocr_model.predict(target_images)
    target_predictions_classes = np.argmax(target_predictions, axis=1)
    valid_labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    target_predicted_labels = [valid_labels[i] for i in target_predictions_classes]

    # Convert predicted OCR labels into a string (assuming the labels form words/sentences)
    predicted_text = ''.join(target_predicted_labels)

    # Map extracted labels to the corresponding files
    target_labels['predicted_label'] = [target_predicted_labels[target_images_files.index(os.path.join(target_images_dir, file))] for file in target_labels['file']]

    # Drop rows with missing values in 'line' column
    sentiment_df = sentiment_df.dropna(subset=['line'])

    # Keep rows with valid sentiment values ('Angry', 'Happy', 'Neutral')
    valid_sentiments = ['Angry', 'Happy', 'Neutral']
    sentiment_df = sentiment_df[sentiment_df['sentiment'].isin(valid_sentiments)]

    # Ensure the dataset is not empty after filtering
    if sentiment_df.empty:
        raise ValueError("No data available for training the sentiment analysis model.")

    # Prepare data for Sentiment Analysis model
    X_sentiment = sentiment_df['line'].values
    y_sentiment = sentiment_df['sentiment'].values

    # Vectorize text data using CountVectorizer
    vectorizer = CountVectorizer()
    X_sentiment_vectorized = vectorizer.fit_transform(X_sentiment)

    # Split into training and test sets
    X_train_sentiment, X_test_sentiment, y_train_sentiment, y_test_sentiment = train_test_split(X_sentiment_vectorized, y_sentiment, test_size=0.2, random_state=42)

    # Train Naive Bayes classifier
    sentiment_model = MultinomialNB()
    sentiment_model.fit(X_train_sentiment, y_train_sentiment)

    # Evaluate Sentiment Analysis model
    y_pred_sentiment = sentiment_model.predict(X_test_sentiment)
    print(classification_report(y_test_sentiment, y_pred_sentiment))

    # Predict the sentiment for each extracted label using the sentiment analysis model
    target_labels['predicted_sentiment'] = [sentiment_model.predict(vectorizer.transform([label]))[0] for label in target_labels['predicted_label']]

    print(target_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
              precision    recall  f1-score   support

       Angry       0.50      0.50      0.50         2
       Happy       0.50      0.50      0.50         2
     Neutral       1.00      1.00      1.00         2

    accuracy                           0.67         6
   macro avg       0.67      0.67      0.67         6
weighted avg       0.67      0.67      0.67         6

         file sentiment predicted_label predicted_sentiment
0  line_1.png     Angry               T               Angry
1  line_2.png     Angry               Y               Angry
2  line_3.png     Happy               T               Angry
3  line_4.png     Happy               T               Angry
4  line_5.png   Neutral               T               Angry
5  line_6.png   Neutral               Y               Angry
